In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/cnn-1.zip"

Streaming output truncated to the last 5000 lines.
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4168.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4169.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4170.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4171.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4172.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4174.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4176.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4178.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4179.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4180.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4181.jpg  
  inflating: 1-condition-splited-data/train/Normal/kidney_normal_4183.jpg  
  inflating: 1-condition-splited-data

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Veri setinin bulunduğu dizin
data_dir = "/content/drive/MyDrive/Colab Notebooks/1-condition-splited-data"

# Veri setini yüklemek için ImageDataGenerator oluşturun
datagen = ImageDataGenerator(rescale=1./255)

# Eğitim seti
train_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'  # Eğer sınıf sayısı 2 ise 'binary'
)

# Test seti
test_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'  # Eğer sınıf sayısı 2 ise 'binary'
)

val_generator = datagen.flow_from_directory(
    os.path.join(data_dir, 'validation'),
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'  # Eğer sınıf sayısı 2 ise 'binary'
)

# Eğitim ve test setlerini oluşturun
X_train, y_train = next(train_generator)
X_test, y_test = next(test_generator)
X_val, y_val = next(val_generator)

print("Eğitim seti boyutu:", X_train.shape, y_train.shape)
print("Test seti boyutu:", X_test.shape, y_test.shape)
print("Test seti boyutu:", X_val.shape, y_val.shape)

Found 7200 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Eğitim seti boyutu: (32, 256, 256, 3) (32, 2)
Test seti boyutu: (32, 256, 256, 3) (32, 2)
Test seti boyutu: (32, 256, 256, 3) (32, 2)


In [ ]:
new_y_test = []
new_y_train = []
new_y_val = []

for i in y_test:
  new_y_test.append( 0 if i[0] == 1 else  1)

for i in y_train:
  new_y_train.append( 0 if i[0] == 1 else  1)

for i in y_val:
  new_y_val.append( 0 if i[0] == 1 else  1)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential


# Model oluşturma fonksiyonu
def build_model(hp):
    model = keras.Sequential()

    # Convolutional katman 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Convolutional katman 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Convolutional katman 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Convolutional katman 4
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Flatten katmanı
    model.add(Flatten())

    # Tam bağlı (fully connected) katman
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout katmanı

    # Çıkış katmanı
    model.add(Dense(2, activation='softmax'))
    loss = tf.keras.losses.CategoricalCrossentropy()
    # Modeli derleme
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    return model


# Keras Tuner RandomSearch tanımlama
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    directory='my_dir',
)
import numpy as np
# Modeli eğitim verileri üzerinde araştırma
tuner.search(X_train, y_train, epochs=40, validation_data=(X_val,y_val))

# En iyi modelin özeti
tuner.results_summary()

# En iyi modeli al
#best_model = tuner.get_best_models(num_models=1)[0]

Trial 30 Complete [00h 00m 08s]
val_accuracy: 0.65625

Best val_accuracy So Far: 0.78125
Total elapsed time: 01h 23m 06s
Results summary
Results in my_dir/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 24 summary
Hyperparameters:
conv1_filter: 80
conv1_kernel: 3
dense1_units: 112
learning_rate: 0.0001
Score: 0.78125

Trial 08 summary
Hyperparameters:
conv1_filter: 112
conv1_kernel: 5
dense1_units: 80
learning_rate: 0.001
Score: 0.75

Trial 11 summary
Hyperparameters:
conv1_filter: 96
conv1_kernel: 5
dense1_units: 128
learning_rate: 0.01
Score: 0.71875

Trial 17 summary
Hyperparameters:
conv1_filter: 48
conv1_kernel: 3
dense1_units: 112
learning_rate: 0.001
Score: 0.71875

Trial 4 summary
Hyperparameters:
conv1_filter: 80
conv1_kernel: 3
dense1_units: 64
learning_rate: 0.001
Score: 0.7142857313156128

Trial 26 summary
Hyperparameters:
conv1_filter: 128
conv1_kernel: 5
dense1_units: 32
learning_rate: 0.01
Score: 0.6875

Trial 09 summary
Hyp

In [ ]:
model = keras.Sequential()

# Convolutional katman 1
model.add(Conv2D(80, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Convolutional katman 2
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Convolutional katman 2
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Convolutional katman 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Convolutional katman 4
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Flatten katmanı
model.add(Flatten())

# Tam bağlı (fully connected) katman
model.add(Dense(112, activation='relu'))
model.add(Dropout(0.5))  # Dropout katmanı

# Çıkış katmanı
model.add(Dense(2, activation='softmax'))
loss = tf.keras.losses.CategoricalCrossentropy()
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


In [ ]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_val,y_val))

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 2.6070 - accuracy: 0.4062 - val_loss: 0.6833 - val_accuracy: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 129ms/step - loss: 5.2119 - accuracy: 0.5000 - val_loss: 0.6898 - val_accuracy: 0.5312
Epoch 3/100
1/1 [==============================] - 0s 129ms/step - loss: 2.3893 - accuracy: 0.6250 - val_loss: 0.6875 - val_accuracy: 0.5625
Epoch 4/100
1/1 [==============================] - 0s 126ms/step - loss: 1.4040 - accuracy: 0.7500 - val_loss: 0.6953 - val_accuracy: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 125ms/step - loss: 1.0990 - accuracy: 0.7812 - val_loss: 0.7121 - val_accuracy: 0.5625
Epoch 6/100
1/1 [==============================] - 0s 125ms/step - loss: 0.3329 - accuracy: 0.8750 - val_loss: 0.7129 - val_accuracy: 0.5312
Epoch 7/100
1/1 [==============================] - 0s 126ms/step - loss: 0.4680 - accuracy: 0.8750 - val_loss: 0.7238 - val_accuracy: 0.5312
Epoch 8/100
1/1 

In [ ]:
tf.keras.models.save_model(model,'cnn-1')

In [ ]:
loaded_model = tf.keras.models.load_model('/content/cnn-1')


In [ ]:
!zip -r cnn-1.zip.zip /content/cnn-1

  adding: content/cnn-1/ (stored 0%)
  adding: content/cnn-1/fingerprint.pb (stored 0%)
  adding: content/cnn-1/variables/ (stored 0%)
  adding: content/cnn-1/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/cnn-1/variables/variables.index (deflated 71%)
  adding: content/cnn-1/assets/ (stored 0%)
  adding: content/cnn-1/keras_metadata.pb (deflated 94%)
  adding: content/cnn-1/saved_model.pb (deflated 89%)


In [ ]:
result = loaded_model.predict(X_test)

1/1 [==============================] - 0s 184ms/step


In [ ]:
results = np.argmax(result,axis=1)
labels = np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


accuracy = accuracy_score(labels, results)
precision = precision_score(labels, results)
recall = recall_score(labels, results)
f1 = f1_score(labels, results)
confusion_mat = confusion_matrix(labels, results)
report = classification_report(labels, results)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(report)

Accuracy: 0.59375
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
[[19  0]
 [13  0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      1.00      0.75        19
           1       0.00      0.00      0.00        13

    accuracy                           0.59        32
   macro avg       0.30      0.50      0.37        32
weighted avg       0.35      0.59      0.44        32



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Unde